In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv


In [2]:
from tensorflow.keras.datasets import cifar10
import tensorflow as tf
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from tensorflow import keras
from keras.utils import to_categorical
from keras.models import load_model
from keras.preprocessing import image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,\
                                    ShuffleSplit, train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

In [4]:
train_df = pd.read_csv('../input/titanic/train.csv')
test_df = pd.read_csv('../input/titanic/test.csv')
gender_submission = pd.read_csv('../input/titanic/gender_submission.csv')
IDs = test_df['PassengerId']
target = train_df.pop('Survived')
data = pd.concat([train_df, test_df], sort=False)
print(f'Train shape: {train_df.shape}\nTest shape: {test_df.shape}\nAll data shape: {data.shape}\
                                                                \nTarget shape: {target.shape}')

Train shape: (891, 11)
Test shape: (418, 11)
All data shape: (1309, 11)                                                                
Target shape: (891,)


In [5]:
data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Pclass       1309 non-null   int64  
 2   Name         1309 non-null   object 
 3   Sex          1309 non-null   object 
 4   Age          1046 non-null   float64
 5   SibSp        1309 non-null   int64  
 6   Parch        1309 non-null   int64  
 7   Ticket       1309 non-null   object 
 8   Fare         1308 non-null   float64
 9   Cabin        295 non-null    object 
 10  Embarked     1307 non-null   object 
dtypes: float64(2), int64(4), object(5)
memory usage: 122.7+ KB


In [7]:
data.drop(['Cabin', 'PassengerId', 'Ticket'], axis=1, inplace=True)

In [8]:
data['Embarked'].value_counts(normalize=True)

S    0.699311
C    0.206580
Q    0.094109
Name: Embarked, dtype: float64

In [9]:
data['Age'].fillna(data['Age'].mean(), inplace=True)
data['Fare'].fillna(data['Fare'].mean(), inplace=True)
data['Embarked'].fillna('S', inplace=True)

In [10]:
data['Name'] = data['Name'].str.split(', ', expand=True)[1].\
                                str.split('.', expand=True)[0]

In [11]:
data['Name'].value_counts()

Mr              757
Miss            260
Mrs             197
Master           61
Rev               8
Dr                8
Col               4
Major             2
Ms                2
Mlle              2
Lady              1
the Countess      1
Jonkheer          1
Dona              1
Sir               1
Mme               1
Don               1
Capt              1
Name: Name, dtype: int64

In [12]:
names = (data['Name'].value_counts() <= 8)


In [13]:
data['Name'] = data['Name'].apply(lambda x: 'Other' if names.loc[x] == True else x)

In [14]:
data['Name'].value_counts()

Mr        757
Miss      260
Mrs       197
Master     61
Other      34
Name: Name, dtype: int64

In [15]:
data['Name'] = data['Name'].map({'Mr': 0,
                                 'Miss': 1,
                                 'Mrs': 2,
                                 'Master': 3,
                                 'Other': 4})

data['Sex'] = data['Sex'].map({'male': 0,
                              'female': 1})

data['Embarked'] = data['Embarked'].map({'S': 0,
                                        'C': 1,
                                        'Q': 2})
data = data.astype('int')

In [16]:
data['Family_size'] = data['SibSp'] + data['Parch'] + 1 
data['Alone'] = 1
data['Alone'].loc[data['Family_size'] > 1] = 0
data['Age_range'] = 0
data['Age_range'].loc[data['Age'] <= 20] = 0 
data['Age_range'].loc[data['Age'] > 20] = 1 
data['Age_range'].loc[data['Age'] > 40] = 2
data['Age_range'].loc[data['Age'] > 60] = 3 

In [17]:
data = pd.get_dummies(data, columns=['Alone', 'Family_size'])

In [18]:
data.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Age_range,Alone_0,Alone_1,Family_size_1,Family_size_2,Family_size_3,Family_size_4,Family_size_5,Family_size_6,Family_size_7,Family_size_8,Family_size_11
0,3,0,0,22,1,0,7,0,1,1,0,0,1,0,0,0,0,0,0,0
1,1,2,1,38,1,0,71,1,1,1,0,0,1,0,0,0,0,0,0,0
2,3,1,1,26,0,0,7,0,1,0,1,1,0,0,0,0,0,0,0,0
3,1,2,1,35,1,0,53,0,1,1,0,0,1,0,0,0,0,0,0,0
4,3,0,0,35,0,0,8,0,1,0,1,1,0,0,0,0,0,0,0,0


In [19]:
train_df = data[:891]
test_df = data[891:]

In [20]:
X_train, X_test, y_train, y_test = train_test_split(train_df, 
                                                    target, 
                                                    test_size=0.2,
                                                    random_state=18)

In [21]:
y_train = y_train.to_numpy()

In [22]:
X_train = X_train.to_numpy()

In [23]:
X_train.shape

(712, 20)

In [24]:
model = Sequential()
model.add(Dense(1024,activation='relu',kernel_initializer='he_uniform', input_shape=X_train.shape))
model.add(Dense(512,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(256,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(256,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(256,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(256,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(128,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(128,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(128,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(64,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(32,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(1,activation='sigmoid'))

In [25]:
from keras.optimizers import Adam

In [26]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss = 'binary_crossentropy', metrics=["binary_accuracy"])

In [27]:
model.fit(X_train,y_train, batch_size=16, epochs=500, validation_data=(X_test, y_test))

Epoch 1/500
45/45 [==============================] - 1s 16ms/step - loss: 0.8449 - binary_accuracy: 0.6306 - val_loss: 0.6741 - val_binary_accuracy: 0.6425
Epoch 2/500
45/45 [==============================] - 0s 9ms/step - loss: 0.8067 - binary_accuracy: 0.6404 - val_loss: 0.5032 - val_binary_accuracy: 0.7709
Epoch 3/500
45/45 [==============================] - 0s 9ms/step - loss: 0.9248 - binary_accuracy: 0.6531 - val_loss: 0.5586 - val_binary_accuracy: 0.7654
Epoch 4/500
45/45 [==============================] - 0s 9ms/step - loss: 0.7353 - binary_accuracy: 0.6601 - val_loss: 0.5352 - val_binary_accuracy: 0.7821
Epoch 5/500
45/45 [==============================] - 0s 8ms/step - loss: 0.6950 - binary_accuracy: 0.6854 - val_loss: 0.5838 - val_binary_accuracy: 0.7039
Epoch 6/500
45/45 [==============================] - 0s 9ms/step - loss: 0.7014 - binary_accuracy: 0.6643 - val_loss: 0.5193 - val_binary_accuracy: 0.7039
Epoch 7/500
45/45 [==============================] - 0s 8ms/step - lo

In [28]:
predict = model.predict_classes(test_df)

In [29]:
from itertools import chain

In [30]:
predict = list(chain.from_iterable(predict))

In [31]:
my_submission = pd.DataFrame({'PassengerId': IDs, 'Survived': predict})
my_submission.to_csv('submission3.csv', index=False)